# Vizualizing BigQuery data in a Jupyter notebook

[BigQuery](https://cloud.google.com/bigquery/docs/) is a petabyte-scale analytics data warehouse that you can use to run SQL queries over vast amounts of data in near realtime.

Data visualization tools can help you make sense of your BigQuery data and help you analyze the data interactively. You can use visualization tools to help you identify trends, respond to them, and make predictions using your data. In this tutorial, you use the BigQuery Python client library and pandas in a Jupyter notebook to visualize data.

We’ll use Airline Ontime Performance data, a 70 million row data set from the U.S. Bureau of Transportation statistics, that is available to all users in BigQuery as the airline_ontime_data.flights data set.

## Using BigQuery magic
The following example shows how to invoke the magic (`%%bigquery`), and how to pass in a standard SQL query in the body of the code cell. The results are displayed below the input cell as a pandas [`DataFrame`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [ ]:
project_id = 'your-project-id' # change to your project id

As the query job is running, status messages below the cell update with the query job ID and the amount of time the query has been running. By default, this output is erased and replaced with the results of the query. If you pass the `--verbose` flag, the output will remain below the cell after query completion. Run the following sell to see what happens.

In [ ]:
%%bigquery --verbose

SELECT 
    CAST(date AS DATE) AS flight_date, 
    FORMAT_DATE('%b',  CAST(date AS DATE)) AS flight_month, 
    EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day, 
    FORMAT_DATE('%a',  CAST(date AS DATE)) AS flight_day_of_week, 
    airline,
    departure_airport,
    arrival_airport,
    CAST(SUBSTR(LPAD(CAST(departure_schedule AS STRING), 4, '0'), 0, 2) AS INT64) AS departure_schedule_hour, 
    CAST(SUBSTR(LPAD(CAST(departure_schedule AS STRING), 4, '0'), 3, 2) AS INT64) AS departure_schedule_minute, 
    CASE 
        WHEN departure_schedule BETWEEN 600 AND 900 THEN '[6:00am - 9:00am]'
        WHEN departure_schedule BETWEEN 900 AND 1200 THEN '[9:00am - 12:pm]'
        WHEN departure_schedule BETWEEN 1200 AND 1500 THEN '[12:00pm - 3:00pm]'
        WHEN departure_schedule BETWEEN 1500 AND 1800 THEN '[3:00pm - 6:00pm]'
        WHEN departure_schedule BETWEEN 1800 AND 2100 THEN '[6:00pm - 9:00pm]'
        WHEN departure_schedule BETWEEN 2100 AND 2400 THEN '[9:00pm - 12:00am]'
        ELSE '[12:00am - 6:00am]'
    END AS departure_time_slot,
    departure_delay,
    arrival_delay
FROM 
    `bigquery-samples.airline_ontime_data.flights`
LIMIT 5

The following cell runs the same query, but this time the results are saved to a variable. The variable name, `total_flights`, is given as an argument to the `%%bigquery`. The results can then be used for further analysis and visualization. Run the cell and check out the difference.

In [ ]:
%%bigquery --verbose total_flights

SELECT 
    CAST(date AS DATE) AS flight_date, 
    FORMAT_DATE('%b',  CAST(date AS DATE)) AS flight_month, 
    EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day, 
    FORMAT_DATE('%a',  CAST(date AS DATE)) AS flight_day_of_week, 
    airline,
    departure_airport,
    arrival_airport,
    CAST(SUBSTR(LPAD(CAST(departure_schedule AS STRING), 4, '0'), 0, 2) AS INT64) AS departure_schedule_hour, 
    CAST(SUBSTR(LPAD(CAST(departure_schedule AS STRING), 4, '0'), 3, 2) AS INT64) AS departure_schedule_minute, 
    CASE 
        WHEN departure_schedule BETWEEN 600 AND 900 THEN '[6:00am - 9:00am]'
        WHEN departure_schedule BETWEEN 900 AND 1200 THEN '[9:00am - 12:pm]'
        WHEN departure_schedule BETWEEN 1200 AND 1500 THEN '[12:00pm - 3:00pm]'
        WHEN departure_schedule BETWEEN 1500 AND 1800 THEN '[3:00pm - 6:00pm]'
        WHEN departure_schedule BETWEEN 1800 AND 2100 THEN '[6:00pm - 9:00pm]'
        WHEN departure_schedule BETWEEN 2100 AND 2400 THEN '[9:00pm - 12:00am]'
        ELSE '[12:00am - 6:00am]'
    END AS departure_time_slot,
    departure_delay,
    arrival_delay
FROM 
    `bigquery-samples.airline_ontime_data.flights`
LIMIT 50000

The next cell uses the pandas `DataFrame.describe` method to visualize the query results in the variable as a bar chart. See the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) to learn more about data visualization with pandas.

In [ ]:
total_flights.describe()

## Exercise 1: Which Airline has the most delays (on average)

For this exercise you need to do:
+ First write a SQL query that GROUPS the average delays by airline. For this query use data from the years 2008-2012
+ Take the results of the query and plot the average delays per airline in a bar charts. 

Answers these questions:
+ Which Airline has the highest average delays?
+ Do you notice anything strange?|

In [ ]:
%%bigquery --verbose avg_delays


In [6]:
# write here the code to plot the bar charts

## Using Python to query BigQuery data

Magic commands allow you to use minimal syntax to interact with BigQuery. Behind the scenes, `%%bigquery` uses the BigQuery Python client library to run the given query, convert the results to a pandas `Dataframe`, optionally save the results to a variable, and finally display the results. Using the BigQuery Python client library directly instead of through magic commands gives you more control over your queries and allows for more complex configurations. The library's integrations with pandas enable you to combine the power of declarative SQL with imperative code (Python) to perform interesting data analysis, visualization, and transformation tasks.

To use the BigQuery Python client library, start by importing the library and initializing a client. The BigQuery client is used to send and receive messages from the BigQuery API.

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

In [ ]:
sql = """
SELECT 
    airline,
    AVG(arrival_delay) AS delay 
FROM 
    (SELECT 
    CAST(date AS DATE) AS flight_date, 
    FORMAT_DATE('%b',  CAST(date AS DATE)) AS flight_month, 
    EXTRACT(DAY FROM CAST(date AS DATE)) AS flight_day, 
    EXTRACT(YEAR FROM CAST(date AS DATE)) AS flight_year,
    FORMAT_DATE('%a',  CAST(date AS DATE)) AS flight_day_of_week,
    airline, 
    arrival_delay
    FROM `bigquery-samples.airline_ontime_data.flights`)
WHERE flight_year BETWEEN 2008 AND 2012
GROUP BY airline
ORDER BY delay DESC
"""
df = client.query(sql).to_dataframe()
df.head()

## Using BigQuery to sample your data

Sampling is important when doing Machine Learning. Sampling on large datasets can be challenging and cumbersome. Thats why we will show how to do efficient, fast, and repeatable selection method that works on very large data sets using BigQuery. 

Naively, the easiest way to sample data in BigQuery is to use the built-in random function in a WHERE clause. The RAND() function returns a value between 0–1, so approximately 80% of the rows in the data set will be selected by the query below. This query will process 2.8 GB when run. 

In [ ]:
%%bigquery --verbose

SELECT
  date,
  airline,
  departure_airport,
  departure_schedule,
  arrival_airport,
  arrival_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  RAND() < 0.8
LIMIT 10000

The RAND() function returns a value between 0–1, so approximately 80% of the rows in the data set will be selected by this query. There are some problems with this way of sampling:

+ How do you create a validation and test set using the 20% data not used in the training set?
+ The RAND() is not repeatable since it returns different data everytime you run it. 

## Exercise 2: Scalable and repeatable sampling using BigQuery

For this exercise you need to:
+ Write a query that samples 70% for training 
+ query a query that returns 10% for testing.

Hereby some tips to get you started:
+ Tip: Use the [console](https://console.cloud.google.com/) to write, run and debug your query.  
+ Write a sql query that samples and splits data in a repeatable way. 
+ Create a query that samples 70% of the date.
+ Select the columns you want, this could be: date, airline, departure_airport, departure_schedule, arrival_airport, arrival_delay. 
+ Hint: Have a look at  [FARM_FINGERPRINT()](https://cloud.google.com/bigquery/docs/reference/standard-sql/hash_functions). 

In [ ]:
%%bigquery --verbose


In [ ]:
%%bigquery --verbose



Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

The random sampling topic in this notebook uses examples from an O'Reilly [blog](https://www.oreilly.com/learning/repeatable-sampling-of-data-sets-in-bigquery-for-machine-learning), written by Valliappa Lakshmanan, on random sampling using BigQuery.  